# Market basket analysis

This Market basket analysis help us in discovering the products that are bought together. This information can be used to stratergically place products in the store or stratergically come up with suggestions for customers buying products online 

In [1]:
# import pandas for dataframe
import pandas as pd

# import numpy for computing functionality
import numpy as np

# import seaborn for visualizing data
import seaborn as sns

%matplotlib inline

#importing apriori from mlxtend
from mlxtend.frequent_patterns import apriori

#import association rules from mlxtend
from mlxtend.frequent_patterns import association_rules
import warnings
warnings.filterwarnings("ignore")

In [2]:
#loading the online retail dataset
data = pd.read_excel("Online Retail.xlsx")

# displaying first 5 rows of the dataset
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [3]:
#remove additional spaces
data['Description'] = data['Description'].str.strip()
#remove NA values
data.dropna(axis=0, subset=['InvoiceNo'], inplace=True)
# converting InvoiceNo to string type
data['InvoiceNo'] = data['InvoiceNo'].astype('str')
#remove cancelled orders
data = data[~data['InvoiceNo'].str.contains('C')]

In [4]:
# Market basket analysis for transactions that involve customers from France
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

def create_basket(country_filter):
    basket = (data[data['Country'] == country_filter]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))
    return basket

** Analysing the France market **

In [5]:
country_filter = "France"
france_data = create_basket("France")
france_dataset = france_data.applymap(encode_units)
france_dataset.drop('POSTAGE', inplace=True, axis=1)

In [6]:
# using apriori for frequent dataset mining for france market transactions
frequent_items = apriori(france_dataset, min_support=0.05, use_colnames=True)

In [7]:
# Fetching association rules for the frequency dataset
rules = association_rules(frequent_items, metric="lift", min_threshold=1.2)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE PINK),0.096939,0.102041,0.073980,0.763158,7.478947,0.064088,3.791383
1,(ALARM CLOCK BAKELIKE PINK),(ALARM CLOCK BAKELIKE GREEN),0.102041,0.096939,0.073980,0.725000,7.478947,0.064088,3.283859
2,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED),0.096939,0.094388,0.079082,0.815789,8.642959,0.069932,4.916181
3,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE GREEN),0.094388,0.096939,0.079082,0.837838,8.642959,0.069932,5.568878
4,(ALARM CLOCK BAKELIKE PINK),(ALARM CLOCK BAKELIKE RED),0.102041,0.094388,0.073980,0.725000,7.681081,0.064348,3.293135


From the table we can notice that the probability of product ALARM CLOCK BAKELIKE PINK with ALARM CLOCK BAKELIKE GREEN being purchased together(support) is 7.3%. The probability of these two products purchased together is 7.3 %. Since the lift score > 1 for these two products, we can make use of ALARM CLOCK BAKELIKE PINK to make predictions on ALARM CLOCK BAKELIKE GREEN, since lift score >1 implies posistive impact. This rule can be incorrect 3.28 times more often if this association rule was purely a random chance.

**Analysing the France market**

In [8]:
# Market basket analysis for transactions that involve customers from Germany
germany_data = create_basket("Germany")
germany_dataset = germany_data.applymap(encode_units)
germany_dataset.drop('POSTAGE', inplace=True, axis=1)

In [9]:
# using apriori for frequent dataset mining for germany market transactions
frequent_itemset = apriori(germany_dataset, min_support=0.05, use_colnames=True)

In [10]:
# Fetching association rules for the frequency dataset
rules = association_rules(frequent_itemset, metric="lift", min_threshold=1.2)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(PLASTERS IN TIN CIRCUS PARADE),(PLASTERS IN TIN WOODLAND ANIMALS),0.115974,0.137856,0.067834,0.584906,4.242887,0.051846,2.076984
1,(PLASTERS IN TIN WOODLAND ANIMALS),(PLASTERS IN TIN CIRCUS PARADE),0.137856,0.115974,0.067834,0.492063,4.242887,0.051846,1.740427
2,(PLASTERS IN TIN CIRCUS PARADE),(ROUND SNACK BOXES SET OF 4 FRUITS),0.115974,0.157549,0.050328,0.433962,2.754455,0.032057,1.488330
3,(ROUND SNACK BOXES SET OF 4 FRUITS),(PLASTERS IN TIN CIRCUS PARADE),0.157549,0.115974,0.050328,0.319444,2.754455,0.032057,1.298977
4,(PLASTERS IN TIN CIRCUS PARADE),(ROUND SNACK BOXES SET OF4 WOODLAND),0.115974,0.245077,0.056893,0.490566,2.001685,0.028470,1.481887


From the table we can notice that the probability of product PLASTERS IN TIN CIRCUS PARADE with PLASTERS IN TIN WOODLAND ANIMALS being purchased together(support) is 6.7%. The probability of these two products purchased together is 58.4 %. Since the lift score > 1 for these two products, we can make use of PLASTERS IN TIN CIRCUS PARADE to make predictions on PLASTERS IN TIN WOODLAND ANIMALS, since lift score > 1 implies posistive impact. This rule can be incorrect 2.07 times more often if this association rule was purely a random chance.

In [11]:
# Market basket analysis for transactions that involve customers from United Kingdom
uk_data = create_basket("United Kingdom")
uk_sets = uk_data.applymap(encode_units)
uk_sets.drop('POSTAGE', inplace=True, axis=1)

In [12]:
# using apriori for frequent dataset mining for UK market transactions
frequent_itemset = apriori(uk_sets, min_support=0.05, use_colnames=True)
# Fetching association rules for the frequency dataset
rules = association_rules(frequent_itemset, metric="lift", min_threshold=0.8)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction


There are no rules for united kingdom for this filter with the specified support value so we will try few other filters.

In [14]:
data_uk = data[data['Country']=="United Kingdom"]

In [15]:
new_uk_ds = (data_uk[data_uk['Quantity']<10]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

In [16]:
uk_ds = new_uk_ds.applymap(encode_units)
uk_ds.drop('POSTAGE', inplace=True, axis=1)

In [17]:
# using apriori for frequent dataset mining for UK market transactions on a different filter
frequent_itemsets = apriori(uk_ds, min_support=0.03, use_colnames=True)
# Fetching association rules for the frequency dataset
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=0.5)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED),0.049209,0.053694,0.030243,0.614583,11.446065,0.027601,2.455281
1,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE GREEN),0.053694,0.049209,0.030243,0.563246,11.446065,0.027601,2.176948
2,(GREEN REGENCY TEACUP AND SAUCER),(PINK REGENCY TEACUP AND SAUCER),0.052092,0.038252,0.030884,0.592866,15.498926,0.028891,2.362239
3,(PINK REGENCY TEACUP AND SAUCER),(GREEN REGENCY TEACUP AND SAUCER),0.038252,0.052092,0.030884,0.807370,15.498926,0.028891,4.920879
4,(ROSES REGENCY TEACUP AND SAUCER),(GREEN REGENCY TEACUP AND SAUCER),0.051259,0.052092,0.036907,0.720000,13.821697,0.034236,3.385386


From the table we can notice that the probability of product ALARM CLOCK BAKELIKE RED with ALARM CLOCK BAKELIKE GREEN being purchased together(support) is 3.02%. The probability of these two products purchased together is 56.3 %. Since the lift score > 1 for these two products, we can make use of ALARM CLOCK BAKELIKE RED to make predictions on ALARM CLOCK BAKELIKE GREEN, since lift score > 1 implies posistive impact. This rule can be incorrect 2.17 times more often if this association rule was purely a random chance.

In [ ]:
# we can get association rule using fp-growth algorithm on uk dataset
import pyfpgrowth
patterns = pyfpgrowth.find_frequent_patterns(uk_ds, 2)
rules = pyfpgrowth.generate_association_rules(patterns, 0.7)
rules.head()

In [ ]:
# similarly we are fetching association rules using fp-growth algorithm on france dataset
import pyfpgrowth
patterns = pyfpgrowth.find_frequent_patterns(france_dataset, 2)
rules = pyfpgrowth.generate_association_rules(patterns, 0.7)
rules.head()

In [ ]:
# Association rules using fp-growth algorithm on germany datset
import pyfpgrowth
patterns = pyfpgrowth.find_frequent_patterns(germany_dataset, 2)
rules = pyfpgrowth.generate_association_rules(patterns, 0.7)
rules.head()